In [107]:
%matplotlib inline
import os, io, json, urllib, numpy as np, codecs, math, collections
import glob, pandas as pd
pd.options.display.max_rows = 999
import pickle

In [35]:
this_month='1711'
data_path='../data/'+this_month
last_year=6

In [36]:
def h_(citation):
    citation=list(citation)
    h=0
    citation.sort(reverse=True)
    for i in range(1,len(citation)+1):
        if citation[i-1] >= i:
            h=i
    return h

In [37]:
df=pd.read_csv(data_path+'/cleaned.csv')
df.shape

(7337746, 15)

In [38]:
#df['year']=df['year'].str[1:5].astype(int)
df=df[df.year<2017]
df.shape

(7337746, 15)

In [39]:
df2=df
df2['@id']=df2.main_issn

In [40]:
dg=df2.groupby(['@id','year']).agg({'count':sum,'citation':sum}).reset_index()
dg['impact']=dg['citation'].div(dg['count']).round(2)
dg.shape

(98146, 5)

In [41]:
dg_h=df2[['@id','citation']].groupby(['@id']).agg(lambda x:tuple(x)).reset_index()
dg_h['h']=dg_h.citation.apply(lambda x: h_(x))
dg_h.shape

(39542, 3)

In [42]:
dg_i=df2.groupby(['@id']).agg({'count':sum,'citation':sum}).reset_index()
dg_i['impact']=dg_i['citation'].div(dg_i['count']).round(2)
dg_i.shape

(39542, 4)

In [43]:
dg_flat=dg.pivot(index='@id',columns='year')
dg_flat.columns = [col[0]+'_'+str(col[1])[3] for col in dg_flat.columns.values]
dg_flat=dg_flat.reset_index()

In [44]:
dg_merge=dg_flat.merge(dg_i[['@id','count','citation','impact']])\
.merge(dg_h[['@id','h']]).fillna(0,downcast='infer')
dg_merge.shape

(39542, 14)

In [45]:
work_batches=dg_merge.shape[0]/10000
for s in range(work_batches+1):
    dic=[]
    for index, i in dg_merge[10000*s:10000*(s+1)].iterrows():
        journal={}
        journal['@id']=i['@id']
        for y in range(4,last_year): 
            journal['e:count_'+str(y)]=i['count_'+str(y)]
            journal['e:citation_'+str(y)]=i['citation_'+str(y)]
            journal['e:impact_'+str(y)]=round(i['impact_'+str(y)],2)
        journal['e:count']=i['count']
        journal['e:citation']=i['citation']
        journal['e:impact']=round(i['impact'],2)
        journal['e:h']=i['h']
        if journal not in dic:
            dic.append(journal)
    with open('../jsonld/'+this_month+'/'+str(s)+'_from_works.jsonld', 'w') as outfile:
        json.dump(dic, outfile)

In [46]:
with open(data_path +'/journals.pkl', 'rb') as f:
    dic=pickle.load(f)

In [47]:
unique_subjects=[]
for j in dic:
    for s in j['e:subjects']: 
        if s not in unique_subjects: 
            unique_subjects.append(s)
len(unique_subjects)  

330

In [48]:
pd.DataFrame(unique_subjects).to_csv('../store/unique_subjects.csv',header=False,index=False,encoding='utf-8')

In [49]:
main_issns=pd.read_csv('../store/main_issns.csv',header=None)
main_issns=main_issns[0].values

In [50]:
with open('..\web\cats.json') as data_file:    
    cat_subject=json.load(data_file)

In [51]:
sub_cat={}
for cat in cat_subject.keys():
    for sub in cat_subject[cat].split(';'):
        sub_cat[sub]=cat

In [52]:
dic_new=[]; i=0
for j in dic:
    i=i+1;
    if i%1000==0: print i/1000,
    j_new=j
    j_new['e:category']=[]
    for subject in j['e:subjects']:
        cat=sub_cat[subject]
        if cat not in j_new['e:category']:
            j_new['e:category'].append(sub_cat[subject])
    if j['e:issn'][0] in main_issns:        
        j_new['@id']=j['e:issn'][0]
        dic_new.append(j_new)    
    elif len(j['e:issn'])>1:
        if j['e:issn'][1] in main_issns:
            j_new['@id']=j['e:issn'][1]
            dic_new.append(j_new)  
len(dic_new)        

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53

39015

In [56]:
doaj_df = pd.read_csv('../store/doaj_20171126_1530_utf8.csv')

In [57]:
doaj_print=doaj_df['Journal ISSN (print version)'].values
doaj_online=doaj_df['Journal EISSN (online version)'].values
doaj_issns=np.concatenate([doaj_print,doaj_online])
doaj_issns=list(set(doaj_issns))

In [58]:
dic_open=[]
i=0
cat=0
for journal in dic_new:
    for issn in journal['e:issn']:
        if issn in doaj_issns:
            i+=1 
            if len(journal['e:category']) >0:
                cat+=1
            journal['e:open']=1
            journal['e:road_url']=doaj_issns[doaj_issns.index(issn)]
            dic_open.append(journal)
            break    
i

6654

In [59]:
cat

2133

In [60]:
journal_batches=len(dic_open)/10000
for s in range(journal_batches+1):
    dic_chunk=dic_open[10000*s:10000*(s+1)]
    with open('../jsonld/'+this_month+'/'+str(s)+'_from_journals.jsonld', 'w') as outfile:
        json.dump(dic_chunk, outfile)

In [93]:
doaj_df.Subjects.fillna('',inplace=True)

In [205]:
l=doaj_df.Subjects.values

In [247]:
subj_l=[]
for subjs in l:
    if len(subjs)>0:
        subj_l=subj_l+subjs.split('|')
subj_l=[subj.strip() for subj in subj_l]

In [581]:
dic={
    "Agriculture":"Agricultural and Biological Sciences",
    "Auxiliary sciences of history":"Arts and Humanities",
    "Bibliography. Library science. Information resources":"Social Sciences",
    "Education":"Social Sciences",
    "Fine Arts":"Arts and Humanities",
    "General Works":"Multidisciplinary",
    "Geography. Anthropology. Recreation":{
        "":"Social Sciences",
        "Environmental sciences":"Environmental Science",
        },
    "History (General) and history of Europe":"Arts and Humanities",
    "History America":"Arts and Humanities",
    "Language and Literature":"Arts and Humanities",
    "Law":"Social Sciences",
    "Medicine":{
        "":"Medicine",
        "Dentistry":"Dentistry",
        "Nursing":"Nursing",
        "Therapeutics. Pharmacology":"Pharmacology, Toxicology and Pharmaceutics",
        "Pediatrics":["Nursing","Medicine"]
        },
    "Music and books on Music":"Arts and Humanities",
    "Naval Science":"Engineering",
    "Philosophy. Psychology. Religion":{
        "":"Arts and Humanities",
        "Psychology":"Psychology",
        },
    "Political science":"Social Science",
    "Science":{
        "Astronomy":"Physics and Astronomy",
        "Biology (General)":{
            "":"Agricultural and Biological Sciences",
            "Genetics":"Biochemistry, Genetics and Molecular Biology",
            "Ecology":["Agricultural and Biological Sciences","Environmental Science"],
            "Cytology":"Genetics and Molecular Biology",
            "Evolution":"Agricultural and Biological Sciences",
            },
        "Botany":"Agricultural and Biological Sciences",
        "Chemistry":"Chemistry",
        "Geology":"Earth and Planetary Sciences",
        "Human anatomy":"Medicine",
        "Mathematics":"Mathematics",
        "Microbiology":"Immunology and Microbiology",
        "Natural history (General)":"Agricultural and Biological Sciences",
        "Physics":"Physics and Astronomy",
        "Physiology":"Medicine",
        "Science (General)":"Multidisciplinary",
        "Zoology":"Agricultural and Biological Sciences",
        },
    "Social Sciences":{
        "":"Social Sciences",
        "Commerce":"Business, Management and Accounting",
        "Economic history and conditions":"Economics, Econometrics and Finance",
        "Economics as a science":"Economics, Econometrics and Finance",
        "Finance":"Economics, Econometrics and Finance",
        "Statistics":"Decision Science",
        },
    "Technology":{
        "":"Engineering",
        "Chemical technology":"Chemical Engineering",
        "Electrical engineering. Electronics. Nuclear engineering":{
            "Electronics":"Computer Science",
            },
        "Engineering":"Engineering",
        "Technology (General)":{
            "Industrial engineering. Management engineering":{
                "Information technology":["Business, Management and Accounting","Computer Science"],
                "Management information systems":["Business, Management and Accounting","Decision Science"],
                }           
            }
        },
    }

In [582]:
lc={}
for l0 in dic.keys():
    if isinstance(dic[l0],dict):
        for l1 in dic[l0].keys():
            if isinstance(dic[l0][l1],dict):
                for l2 in dic[l0][l1].keys():
                    if isinstance(dic[l0][l1][l2],dict):
                        for l3 in dic[l0][l1][l2].keys():
                            lc[l0+': '+l1+': '+l2+': '+l3]=dic[l0][l1][l2][l3]
                    else: 
                        if l2=='':
                            lc[l0+': '+l1]=dic[l0][l1][l2]
                        else:
                            lc[l0+': '+l1+': '+l2]=dic[l0][l1][l2]
            else:
                if l1=='':
                    lc[l0]=dic[l0][l1]
                else:
                    lc[l0+': '+l1]=dic[l0][l1]

    else:
        lc[l0]=dic[l0]


In [583]:
subj_clean=[]
for subj in subj_l:
    in_dic=0
    l=subj.split(':')
    for i in range(len(l)):
        key=':'.join(l[0:len(l)-i])
        if key in lc.keys():
            in_dic=1
            if isinstance(lc[key],basestring):
                subj_clean.append(lc[key])
            else:
                for s in lc[key]:
                    subj_clean.append(s)
            break
    if in_dic==0:
        subj_clean.append(subj)

counter=collections.Counter(subj_clean)
counter.most_common()

[('Social Sciences', 3242),
 ('Medicine', 2503),
 ('Arts and Humanities', 2146),
 ('Engineering', 1061),
 ('Agricultural and Biological Sciences', 1044),
 ('Business, Management and Accounting', 515),
 ('Mathematics', 405),
 ('Multidisciplinary', 367),
 ('Social Science', 265),
 ('Environmental Science', 227),
 ('Physics and Astronomy', 208),
 ('Psychology', 203),
 ('Chemistry', 189),
 ('Chemical Engineering', 175),
 ('Computer Science', 163),
 ('Nursing', 145),
 ('Science', 111),
 ('Earth and Planetary Sciences', 107),
 ('Economics, Econometrics and Finance', 105),
 ('Pharmacology, Toxicology and Pharmaceutics', 103),
 ('Dentistry', 90),
 ('Biochemistry, Genetics and Molecular Biology', 73),
 ('Immunology and Microbiology', 60),
 ('Decision Science', 33),
 ('Genetics and Molecular Biology', 31),
 ('Military Science', 17)]

In [580]:
len(counter)

27